# LLM 集成

## 安装指南

Bridgic 使用模块化安装策略——**仅安装您所需的组件**。

每个模型集成都作为单独的包提供，因此您可以最小化依赖关系并保持环境的简洁。

```bash
# 对于一般的 OpenAI 兼容 API，仅支持基本聊天接口（Groq、Together AI 等）
pip install bridgic-llms-openai-like

# 对于 OpenAI 模型（GPT-4、GPT-3.5 等）
pip install bridgic-llms-openai

# 对于 vLLM 服务器部署
pip install bridgic-llms-vllm
```

|         包               | `BaseLlm` | `StructuredOutput` | `ToolSelection` |
|:-------------------------:|:---------:|:------------------:|:---------------:|
| `bridgic-llms-openai-like`|     ✅     |         ❌          |        ❌        |
| `bridgic-llms-openai`     |     ✅     |         ✅          |        ✅        |
| `bridgic-llms-vllm`       |     ✅     |         ✅          |        ✅        |

## LLM 使用

本节演示了在 Bridgic 中使用模型的完整生命周期，从初始化到高级功能。

### 1. 初始化

模型初始化非常简单，并且在所有提供者中遵循一致的模式。

In [2]:
import os
from dotenv import load_dotenv
from bridgic.llms.openai import OpenAILlm, OpenAIConfiguration

load_dotenv()

_api_key = os.environ.get("OPENAI_API_KEY")

llm = OpenAILlm(
    api_key=_api_key,
)

config = OpenAIConfiguration(
    model="gpt-4o",
    temperature=0.7,
    max_tokens=2000,
)

llm = OpenAILlm(
    api_key=_api_key,
    configuration=config,
    timeout=30.0,
)

### 2. 基础接口

所有在 Bridgic 中的 LLM 提供者都实现了 [`BaseLlm`](../../../../reference/bridgic-core/bridgic/core/model/#bridgic.core.model.BaseLlm) 抽象类，该类定义了基本的 `chat`/`stream` 接口及其异步变体。

#### 2.1 聊天

获取模型完整响应的最基本接口：

In [6]:
from bridgic.core.model.types import Message, Role

# Create messages
messages = [
    Message.from_text("You are a helpful assistant.", role=Role.SYSTEM),
    Message.from_text("What is the capital of France?", role=Role.USER),
]

# Get response
response = llm.chat(
    messages=messages,
    model="gpt-4o",
    temperature=0.7,
)

print(response.message.content)


The capital of France is Paris.


#### 2.2 流式传输

用于实时响应生成：

In [8]:
# Stream response chunks
for chunk in llm.stream(messages=messages, model="gpt-4o"):
    print(chunk.delta, end="|", flush=True)  # Print each chunk as it arrives

The| capital| of| France| is| Paris|.|

### 3. 高级协议

通过可选协议提供高级接口，供应商可以根据其能力实现这些协议。

#### 3.1 结构化输出 ([`StructuredOutput`](../../../../reference/bridgic-core/bridgic/core/model/protocols/#bridgic.core.model.protocols.StructuredOutput) 协议)

生成符合特定模式或格式的输出：

In [14]:
from pydantic import BaseModel, Field
from bridgic.core.model.protocols import PydanticModel, JsonSchema

# Option 1: Using Pydantic Models
class MathProblemSolution(BaseModel):
    """Solution to a math problem with reasoning"""
    reasoning: str = Field(description="Step-by-step reasoning")
    answer: int = Field(description="Final numerical answer")

messages = [
    Message.from_text("What is 15 * 23?", role=Role.USER)
]

# Get structured output
solution = llm.structured_output(
    messages=messages,
    constraint=PydanticModel(model=MathProblemSolution),
    model="gpt-4o",
)

print(f"REASONING:\n\n{solution.reasoning}\n")
print(f"ANSWER: {solution.answer}\n")

REASONING:

15 multiplied by 23 can be broken down into smaller, more manageable calculations using the distributive property of multiplication. Here's how:

1. **Break down 23:**
   - 23 can be expressed as 20 + 3.

2. **Apply the distributive property:**
   - 15 * 23 = 15 * (20 + 3)
   - According to the distributive property, this can be expanded to:
     - 15 * 20 + 15 * 3

3. **Calculate the individual products:**
   - **15 * 20**
     - 15 * 2 = 30
     - Append a zero (since you are multiplying by 20, which is 10 times 2):
     - 15 * 20 = 300
   
   - **15 * 3**
     - 15 * 3 = 45

4. **Add the two results together:**
   - 300 + 45 = 345

Thus, using the distributive property and breaking down the numbers into simpler parts, we find that 15 multiplied by 23 equals 345.

ANSWER: 345



#### 3.2 工具选择 ([`ToolSelection`](../../../../reference/bridgic-core/bridgic/core/model/protocols/#bridgic.core.model.protocols.ToolSelection) 协议)

使模型能够选择和使用工具（函数调用）：

In [15]:
from bridgic.core.model.types import Tool

# Define available tools
tools = [
    Tool(
        name="get_weather",
        description="Get the current weather for a location",
        parameters={
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City name, e.g., 'San Francisco, CA'"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "Temperature unit"
                }
            },
            "required": ["location"]
        }
    ),
    Tool(
        name="calculate",
        description="Perform mathematical calculations",
        parameters={
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "Mathematical expression to evaluate"
                }
            },
            "required": ["expression"]
        }
    )
]

# Model selects appropriate tool
messages = [
    Message.from_text("What's the weather like in Paris?", role=Role.USER)
]

tool_calls, content = llm.select_tool(
    messages=messages,
    tools=tools,
    model="gpt-4o",
    tool_choice="auto",
)

# Process tool calls
for tool_call in tool_calls:
    print(f"Tool: {tool_call.name}")
    print(f"Arguments: {tool_call.arguments}")
    print(f"Call ID: {tool_call.id}")


Tool: get_weather
Arguments: {'location': 'Paris'}
Call ID: call_aLv7xon4zhsNVMcnLmxsGJ3v
